In [0]:
from google.colab import files
import pandas as pd
import io
import re
import numpy
import random
import time, math


uploaded = files.upload()
uploaded2 = files.upload()
uploaded3 = files.upload()
uploaded4 = files.upload()
kanye = uploaded["kanye_verses.txt"].decode("utf-8").lower()
ts = uploaded2["cleaned_taylorLyrics.txt"].decode("utf-8").lower()
sp = uploaded3["Shakespeare.txt"].decode("utf-8").lower()
nw = uploaded4["Nightwish_Lyrics.txt"].decode("utf-8").lower()

Saving kanye_verses.txt to kanye_verses (1).txt


Saving cleaned_taylorLyrics.txt to cleaned_taylorLyrics (1).txt


Saving Shakespeare.txt to Shakespeare (1).txt


Saving Nightwish_Lyrics.txt to Nightwish_Lyrics (1).txt


In [0]:
# this "cleans" the data set to have only lowercase and whitespace

kanye = re.sub('[^a-z ]+',"",kanye)
ts =  re.sub('[^a-z ]+',"",ts)
sp =  re.sub('[^a-z ]+',"",sp)
nw =re.sub('[^a-z ]+',"",nw)
# all letters used in the file, alphabets and whitespace
char_set = sorted(list(set(kanye)))
length = len(char_set)-1
# removes non alphabet, from character set
for i in reversed(range(length)):
  if re.search('[a-z ,]',char_set[i]) == None:
    del char_set[i]


print(char_set)

char_len = len(char_set)
#dictionary mapping from char to int and int to char
char_to_int = dict((c, i) for i, c in enumerate(char_set))
int_to_char = dict((c, i) for i, c in enumerate(char_set))

# splits lyrics by line
def get_text(lyrics):
  split = lyrics.split('\n')
  split2 = [line for line in split if line.strip() != ""]
  text =''
  for line in split2:
    text = text+line
  return text
#dictionary containing all lyrics
texts = dict()
texts["kanye"] = get_text(kanye)
texts["taylorswift"] =get_text(ts)
texts["shakespeare"] = get_text(sp)
texts["nightwish"] = get_text(nw)


[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
# LSTM with the choice to make is bidirectional
class LSTM(nn.Module):
    def __init__(self, x, h, y, n_layers,bi,drop):
        super(LSTM, self).__init__()
        self.x = x
        self.h = h
        self.y = y
        self.n_layers = n_layers
        self.bi = bi
        self.encoder = nn.Embedding(x, h)
        self.lstm = nn.LSTM(h, h, n_layers,bidirectional = bi,dropout=drop)
        if bi ==True:
          self.decoder = nn.Linear(h*2, y)
        else:
          self.decoder = nn.Linear(h, y)
    def forward(self, x, h):

        x = self.encoder(x.view(1, -1))
        if self.bi ==True:
           c0 = torch.zeros(self.n_layers*2, x.size(0), self.h).requires_grad_()
        else:
           c0 = torch.zeros(self.n_layers, x.size(0), self.h).requires_grad_()
       
        y, (h,c0) = self.lstm(x.view(1, 1, -1),(h,c0))
        y = self.decoder(y.view(1, -1))
        
        return y, h

    def init_h(self):
      if self.bi ==True:
        return Variable(torch.zeros(self.n_layers*2, 1, self.h))
      else:
        return Variable(torch.zeros(self.n_layers, 1, self.h))
# GRU with the choice to make is bidirectional
class GRU(nn.Module):
    def __init__(self, x, h, y, n_layers,bi,drop):
        super(GRU, self).__init__()
        self.x = x
        self.h = h
        self.y = y
        self.n_layers = n_layers
        self.bi = bi
        self.encoder = nn.Embedding(x, h)
        self.gru = nn.GRU(h, h, n_layers,bidirectional=bi,dropout=drop)
        if bi ==True:
          self.decoder = nn.Linear(h*2,y)
        else:
          self.decoder = nn.Linear(h,y)
    
    def forward(self, x, h):
      
        x = self.encoder(x.view(1, -1))
        y, h = self.gru(x.view(1, 1, -1), h)
      
        y= self.decoder(y.view(1, -1))
        
        return y, h

    def init_h(self):
        if self.bi ==True:
          return Variable(torch.zeros(self.n_layers*2, 1, self.h))
        else:
          return Variable(torch.zeros(self.n_layers, 1, self.h))



In [0]:
def train_time(start):
    second = time.time() - start
    minute = math.floor(second / 60)
    second -= minute * 60
    return '%dm %ds' % (minute, second)

# batch szie
batch_size = 100
# creates random batch string
def batch(text):
    start = random.randint(0, len(text) - batch_size)
    end = start + batch_size
    return text[start:end+1]

# changes string to tensor 
def char_to_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for i in range(len(string)):
        tensor[i] = char_to_int[string[i]]
    return Variable(tensor)

# gets random training string in tensor
def rand_x(text):    
    data = batch(text)
    x = char_to_tensor(data[:-1]).unsqueeze(1)
    y = char_to_tensor(data[1:]).unsqueeze(1)
   
    return x, y





In [0]:
# evaluates model
def eval(seed, y, temp,decoder):
    h = decoder.init_h()
    seed_tensor = char_to_tensor(seed)
    predict = seed

   
    for i in range(len(seed) - 1):
        _, h = decoder(seed_tensor[i], h)
        x = seed_tensor[-1]
    
    for i in range(y):
        y, h = decoder(x, h)
        
        yp = y.data.view(-1)
        yp = yp.div(temp).exp()

        first = torch.multinomial(yp, 1)[0]
        char = char_set[first]
        predict += char
        x = char_to_tensor(char)

    return predict




def train(x, y,decoder,criterion,opt):
    h = decoder.init_h()
    decoder.zero_grad()
    loss = 0
    for i in range(batch_size):
        out, h = decoder(x[i], h)
      

        loss += criterion(out, y[i])
    loss.backward()
    opt.step()
    avg_loss =  loss.data / batch_size

    return avg_loss


In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def init_model(char_len, h, n_layers,lr,epochs,artist,model,bi,drop):
  text = ""
  if model=="LSTM":
    decoder = LSTM(char_len, h, char_len, n_layers,bi,drop)
  elif model=="GRU":
    decoder= GRU(char_len, h, char_len, n_layers,bi,drop)
  else:
    return None
  opt = torch.optim.Adam(decoder.parameters(), lr=lr)
  criterion = nn.CrossEntropyLoss()
  loss_avg = 0
  text=texts[artist]
  losses = []
  for epoch in range(1, epochs+1):
      
      loss = train(*rand_x(text),decoder,criterion,opt)       
      loss_avg += loss
  
      if epoch % 100 == 0:
          print('[%s (%d %d%%) loss=%.4f model=%s,text=%s,learning rate= %s,bidirectional= %s,dropout =%.2f]' % (train_time(start), epoch, epoch / epochs * 100, loss_avg/100,model,artist,lr,bi,drop))
          # change the seed string here by putting in eval(seed string)
          print(eval('i hope you think ', 100,0.8,decoder), '\n')

      if epoch % 100 == 0:
        losses.append(loss_avg / 100)
        loss_avg = 0
    return decoder
  plt.figure()
  plt.plot(losses)

In [0]:

#set epochs
epochs = 5000
#set number of hidden layers
h = 100
#set number of rnn layers
n_layers = 2
#length of random training example
batch_size = 100
# set learning rate
lr = 0.005
start = time.time()
# set model type
lstm = "LSTM"
gru = "GRU"
# set to false to for non bidirectional
bidirectional = True
# set dropout
drop = 0.2




 24s (100 2%) loss=2.4258 model=LSTM,text=nightwish,learning rate= 0.005,bidirectional= True,dropout =0.20]
i hope you think galleno the the yo diethe sere i dapistowdenting wane a fhe poulof to the s the wilef wileriend sill 

 48s (200 4%) loss=2.1881 model=LSTM,text=nightwish,learning rate= 0.005,bidirectional= True,dropout =0.20]
i hope you think the mistayhfor evere forestile love of wightling ithimich unang lilink the lighgind sose fold thithd 

 12s (300 6%) loss=2.0655 model=LSTM,text=nightwish,learning rate= 0.005,bidirectional= True,dropout =0.20]
i hope you think war hand to a dreary daut me of the farles anc heart doseboneyou so t this tond sefrows iden the me  

 36s (400 8%) loss=1.9898 model=LSTM,text=nightwish,learning rate= 0.005,bidirectional= True,dropout =0.20]
i hope you think touch of the stanity in un one the skeard on an to snout sclows cowers the snompinged gild shis me t 

 0s (500 10%) loss=1.9505 model=LSTM,text=nightwish,learning rate= 0.005,bidirectional= 

In [0]:
# train and eval through init_model(char_len,hidden layer number, number of layers, learning rate, n of epochs, artist type,lstm or gru, bidirectional = false/true, drop = 0.0~1.0)
model = init_model(char_len, h, n_layers,lr,epochs,"taylorswift",lstm,bidirectional,drop=0.2)


In [ ]:
#test model, eval(seed string,length of prediction string,temperature(randomness),trained model)
eval("this is",100,0.8,model)
